In [1]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 50) ## to display more rows
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, ParameterGrid, GridSearchCV, RandomizedSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

## Defining the s3 bucket
s3 = boto3.resource('s3')
bucket_name = 'data-454'
bucket = s3.Bucket(bucket_name)

## Defining the file to be read from s3 bucket
file_key_train = 'Project_1/dmc2010_train.txt'
file_key_test = 'Project_1/dmc2010_class.txt'

bucket_object_train = bucket.Object(file_key_train)
file_object_train = bucket_object_train.get()
file_content_stream_train = file_object_train.get('Body')

bucket_object_test = bucket.Object(file_key_test)
file_object_test = bucket_object_test.get()
file_content_stream_test = file_object_test.get('Body')

## Reading train and test files
train = pd.read_csv(file_content_stream_train, sep = ';')
test = pd.read_csv(file_content_stream_test, sep = ';')

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Feature Engineering of Train

In [2]:
train['feature_1'] = np.where((train['salutation'] == 0) & (train['w0'] >= 1) & (train['deliverydatereal'] != '0000-00-00'), 1, 0) 

train.head()

,customernumber,date,salutation,title,domain,datecreated,newsletter,model,paymenttype,deliverytype,invoicepostcode,delivpostcode,voucher,advertisingdatacode,case,numberitems,gift,entry,points,shippingcosts,deliverydatepromised,deliverydatereal,weight,remi,cancel,used,w0,w1,w2,w3,w4,w5,w6,w7,w8,w9,w10,target90,feature_1
0,41191,2008-12-01,0,0,9,2008-12-01,0,2,2,0,58,NaN,1,NaN,2,2,0,1,0,0,2008-12-03,2008-12-02,737,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1
1,38860,2008-12-16,1,0,4,2008-12-16,0,1,1,1,34,NaN,0,NaN,2,2,0,0,0,0,2008-12-30,2009-02-03,368,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,61917,2008-08-19,0,0,12,2008-08-19,0,1,0,0,51,NaN,0,NaN,1,1,0,0,0,1,2008-09-02,2008-08-28,47,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,40647,2008-06-16,1,0,8,2008-06-16,0,1,0,0,25,NaN,0,NaN,3,2,0,0,0,0,2008-06-17,0000-00-00,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0
4,1347,2008-08-08,0,0,1,2008-08-08,0,1,1,1,41,NaN,0,BR,4,2,0,0,0,0,2008-08-11,2008-08-08,843,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0


In [3]:
pd.crosstab(train['feature_1'], train['target90'])

target90,0,1
feature_1,,
0,18806,4218
1,7571,1833


## Feature Engineering of Test